<a href="https://colab.research.google.com/github/tubagokhan/DeepLearningNLPFoundations/blob/main/POS_tagging_with_Hidden_Markov_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text](https://wisdomml.in/hidden-markov-model-hmm-in-nlp-python/)

## Exploring Treebank Tagged Corpus

In [ ]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

#download the treebank corpus from nltk
nltk.download('treebank')

# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents())

# first tagged sentence
print(wsj[:1])

## Train Test Split

In [ ]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(wsj,test_size=0.3)
print(len(train_set))
print(len(test_set))
print(train_set[:1])

In [ ]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

In [ ]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
# vocabulary
V = set(tokens)
print("Total vocabularies: ",len(V))
# number of tags
T = set([pair[1] for pair in train_tagged_words])
print("Total tags: ",len(T))

'LS': list item marker
'PDT': predeterminer
'SYM': Symbol
'DT': determiner
'-RRB-': comporative adv
'VBZ': verb 3sg pres
'EX': existential ‘there’
'VBP': verb non-3sg-pr
':': 
'WP': wh-pronoun
'TO': to
'MD': modal
'NNPS': proper noun: plu
'IN': preposition
'VBD': verb past tense
'PRP': personal pronoun
':': 
'VBN': verb past participle
'VBG': verb gerund
'$': 
'-NONE-': 
'UH': interjection
'PRP$': possess: pronoun
'WDT': wh-determ.
'JJS': superlative adj
'POS': possessive ending
'#': 
"''": 
'NNS': noun: plural
'JJR': comparative adj
'FW': foreign word
'CD': cardinal number
'VB': verb base
'-LRB-': 
'RB': adverb
'NN': sing or mass noun
'``':
'WP$': wh-posses
'NNP': proper noun: sing.
'WRB': wh-adverb
'JJ': adjective
'RBS': superlative adv
'RBR': comparative adv
'RP': adverb
'CC': coord. Conj.
'.'


## Emission probabilities

In [ ]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

# examples

# large
print("\n", "large")
print(word_given_tag('large', 'JJ')) ## JJ adjective
print(word_given_tag('large', 'VB')) ## VB verb base
print(word_given_tag('large', 'NN'), "\n") ## NN sing or mass noun

# will
print("\n", "will")
print(word_given_tag('will', 'MD')) ## MD modal
print(word_given_tag('will', 'NN'))
print(word_given_tag('will', 'VB'))

# book
print("\n", "book")
print(word_given_tag('book', 'NN'))
print(word_given_tag('book', 'VB'))

## Transition Probabilities

In [ ]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

# examples
print(t2_given_t1(t2='NNP', t1='JJ')) ## NNP proper noun singular , JJ adjective
print(t2_given_t1('NN', 'JJ')) ## NN sing or mass noun, JJ adjective
print(t2_given_t1('NN', 'DT')) ## NN sing or mass noun, DT determiner
print(t2_given_t1('NNP', 'VB')) ## NNP proper noun singular, VB verb base
print(t2_given_t1(',', 'NNP'))
print(t2_given_t1('PRP', 'PRP')) ## PRP personel pronoun
print(t2_given_t1('VBG', 'NNP')) ## VBG verb gerund, NNP proper noun singular

In [ ]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

tags_matrix # transition matrix

In [ ]:
len(tags_matrix)

In [ ]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

In [ ]:
# heatmap of tags matrix
# T(i, j) means P(tag j given tag i)
plt.figure(figsize=(8, 5))
sns.heatmap(tags_df)
plt.show()

In [ ]:
# frequent tags
# filter the df to get P(t2, t1) > 0.5
tags_frequent = tags_df[tags_df>0.5]
plt.figure(figsize=(8, 5))
sns.heatmap(tags_frequent)
plt.show()

## Viterbi Algorithm

In [ ]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

## Evaluating on Test Set

In [ ]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

In [ ]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq)

In [ ]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)

In [ ]:
## Testing
sentence_test = 'Twitter is the best networking social site. Man is a social animal. Data science is an emerging field. Data science jobs are high in demand.'
words = word_tokenize(sentence_test)

start = time.time()
tagged_seq = Viterbi(words)
print(tagged_seq)